### In our last model in file "Music_Generation_Train1.ipynb", we got only 82% accuracy. However, in order to generate melodious music, we need at least 90% accuracy. 
### So, we have loaded the weights of last epoch from our previous model into this model and also we have added 2 extra layers of LSTM here with more LSTM units. 
### Here, we are fine-tuning our old layers and we have added more layers. In short, here we are doing "Transfer Learning" from old to new model.

In [7]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

In [8]:
data_directory = "Data2/"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = 'Data2/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [9]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i] + 1. 
        yield X, Y

In [10]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length), name = "embd_1")) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True, name = "lstm_first"))
    model.add(Dropout(0.2, name = "drp_1"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    model.load_weights("Data/Model_Weights/Weights_80.h5", by_name = True)
    
    return model

In [11]:
def training_model(data, epochs = 100):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("Data2/log.csv", index = False)

In [12]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 87
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embd_1 (Embedding)           (16, 64, 512)             44544     
_________________________________________________________________
lstm_first (LSTM)            (16, 64, 256)             787456    
_________________________________________________________________
drp_1 (Dropout)              (16, 64, 256)             0         
_________________________________________________________________
lstm_2 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_2 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_3 (LSTM)                (16, 64, 256)             525312    
___________________________________________________________

Batch: 113, Loss: 3.204007148742676, Accuracy: 0.146484375
Batch: 114, Loss: 3.059480667114258, Accuracy: 0.1845703125
Batch: 115, Loss: 3.0991525650024414, Accuracy: 0.1796875
Batch: 116, Loss: 3.0916430950164795, Accuracy: 0.171875
Batch: 117, Loss: 3.2151553630828857, Accuracy: 0.1611328125
Batch: 118, Loss: 3.195070266723633, Accuracy: 0.16796875
Batch: 119, Loss: 3.216514825820923, Accuracy: 0.1552734375
Batch: 120, Loss: 3.045205593109131, Accuracy: 0.1708984375
Batch: 121, Loss: 3.091592311859131, Accuracy: 0.1865234375
Batch: 122, Loss: 3.2159454822540283, Accuracy: 0.1513671875
Batch: 123, Loss: 3.1144495010375977, Accuracy: 0.169921875
Batch: 124, Loss: 3.089141368865967, Accuracy: 0.1796875
Batch: 125, Loss: 3.04295015335083, Accuracy: 0.19140625
Batch: 126, Loss: 3.064728260040283, Accuracy: 0.1875
Batch: 127, Loss: 3.0951597690582275, Accuracy: 0.171875
Batch: 128, Loss: 3.0970373153686523, Accuracy: 0.1767578125
Batch: 129, Loss: 3.0180416107177734, Accuracy: 0.21484375
B

Batch: 102, Loss: 2.0772297382354736, Accuracy: 0.4091796875
Batch: 103, Loss: 2.462012767791748, Accuracy: 0.3447265625
Batch: 104, Loss: 2.0995285511016846, Accuracy: 0.408203125
Batch: 105, Loss: 2.1198067665100098, Accuracy: 0.4189453125
Batch: 106, Loss: 2.2290830612182617, Accuracy: 0.3798828125
Batch: 107, Loss: 2.3097949028015137, Accuracy: 0.3681640625
Batch: 108, Loss: 2.306378126144409, Accuracy: 0.3896484375
Batch: 109, Loss: 2.245222806930542, Accuracy: 0.3876953125
Batch: 110, Loss: 2.0956432819366455, Accuracy: 0.3974609375
Batch: 111, Loss: 2.065406560897827, Accuracy: 0.4189453125
Batch: 112, Loss: 2.332167148590088, Accuracy: 0.3701171875
Batch: 113, Loss: 2.300601005554199, Accuracy: 0.40234375
Batch: 114, Loss: 2.2244038581848145, Accuracy: 0.3896484375
Batch: 115, Loss: 2.3302900791168213, Accuracy: 0.390625
Batch: 116, Loss: 2.3000898361206055, Accuracy: 0.3701171875
Batch: 117, Loss: 2.353043556213379, Accuracy: 0.3720703125
Batch: 118, Loss: 2.2597358226776123, 

Batch: 89, Loss: 1.9554060697555542, Accuracy: 0.470703125
Batch: 90, Loss: 1.8432774543762207, Accuracy: 0.4892578125
Batch: 91, Loss: 1.8173580169677734, Accuracy: 0.4873046875
Batch: 92, Loss: 1.8640000820159912, Accuracy: 0.4755859375
Batch: 93, Loss: 1.8082026243209839, Accuracy: 0.494140625
Batch: 94, Loss: 1.8042654991149902, Accuracy: 0.4921875
Batch: 95, Loss: 1.8285658359527588, Accuracy: 0.470703125
Batch: 96, Loss: 1.8833634853363037, Accuracy: 0.4951171875
Batch: 97, Loss: 1.7658987045288086, Accuracy: 0.521484375
Batch: 98, Loss: 1.730473518371582, Accuracy: 0.5458984375
Batch: 99, Loss: 1.6627740859985352, Accuracy: 0.5302734375
Batch: 100, Loss: 1.7470253705978394, Accuracy: 0.5107421875
Batch: 101, Loss: 1.7916525602340698, Accuracy: 0.50390625
Batch: 102, Loss: 1.672360897064209, Accuracy: 0.509765625
Batch: 103, Loss: 1.8947936296463013, Accuracy: 0.4990234375
Batch: 104, Loss: 1.6677639484405518, Accuracy: 0.515625
Batch: 105, Loss: 1.7920942306518555, Accuracy: 0.4

Batch: 78, Loss: 1.6678333282470703, Accuracy: 0.544921875
Batch: 79, Loss: 1.486959457397461, Accuracy: 0.5830078125
Batch: 80, Loss: 1.5420818328857422, Accuracy: 0.5322265625
Batch: 81, Loss: 1.7086398601531982, Accuracy: 0.470703125
Batch: 82, Loss: 1.6847569942474365, Accuracy: 0.4814453125
Batch: 83, Loss: 1.5392353534698486, Accuracy: 0.5556640625
Batch: 84, Loss: 1.596492052078247, Accuracy: 0.5537109375
Batch: 85, Loss: 1.4815654754638672, Accuracy: 0.5703125
Batch: 86, Loss: 1.8170057535171509, Accuracy: 0.486328125
Batch: 87, Loss: 1.6097556352615356, Accuracy: 0.54296875
Batch: 88, Loss: 1.7480754852294922, Accuracy: 0.5087890625
Batch: 89, Loss: 1.7461217641830444, Accuracy: 0.5009765625
Batch: 90, Loss: 1.616797924041748, Accuracy: 0.5361328125
Batch: 91, Loss: 1.5809358358383179, Accuracy: 0.5224609375
Batch: 92, Loss: 1.690337896347046, Accuracy: 0.50390625
Batch: 93, Loss: 1.5827383995056152, Accuracy: 0.5263671875
Batch: 94, Loss: 1.602843165397644, Accuracy: 0.519531

...

Batch: 148, Loss: 0.3481173813343048, Accuracy: 0.8857421875
Batch: 149, Loss: 0.3136559724807739, Accuracy: 0.8984375
Batch: 150, Loss: 0.32375890016555786, Accuracy: 0.8994140625
Batch: 151, Loss: 0.3054651618003845, Accuracy: 0.90234375
Epoch 97/100
Batch: 1, Loss: 0.4352201223373413, Accuracy: 0.857421875
Batch: 2, Loss: 0.34881341457366943, Accuracy: 0.8818359375
Batch: 3, Loss: 0.33467158675193787, Accuracy: 0.8916015625
Batch: 4, Loss: 0.29047903418540955, Accuracy: 0.90625
Batch: 5, Loss: 0.28211942315101624, Accuracy: 0.9013671875
Batch: 6, Loss: 0.29880309104919434, Accuracy: 0.8984375
Batch: 7, Loss: 0.3056877851486206, Accuracy: 0.9052734375
Batch: 8, Loss: 0.3077223598957062, Accuracy: 0.896484375
Batch: 9, Loss: 0.3393809497356415, Accuracy: 0.8857421875
Batch: 10, Loss: 0.3249064087867737, Accuracy: 0.8837890625
Batch: 11, Loss: 0.3200089931488037, Accuracy: 0.8818359375
Batch: 12, Loss: 0.31689584255218506, Accuracy: 0.896484375
Batch: 13, Loss: 0.2731603980064392, Accu

Batch: 135, Loss: 0.3086080849170685, Accuracy: 0.8974609375
Batch: 136, Loss: 0.33330243825912476, Accuracy: 0.9013671875
Batch: 137, Loss: 0.3428748846054077, Accuracy: 0.8837890625
Batch: 138, Loss: 0.3143044114112854, Accuracy: 0.884765625
Batch: 139, Loss: 0.2816968262195587, Accuracy: 0.9052734375
Batch: 140, Loss: 0.3111647665500641, Accuracy: 0.8935546875
Batch: 141, Loss: 0.35334211587905884, Accuracy: 0.876953125
Batch: 142, Loss: 0.36344626545906067, Accuracy: 0.8740234375
Batch: 143, Loss: 0.30670905113220215, Accuracy: 0.8935546875
Batch: 144, Loss: 0.3414687216281891, Accuracy: 0.8720703125
Batch: 145, Loss: 0.3199775815010071, Accuracy: 0.890625
Batch: 146, Loss: 0.3466874361038208, Accuracy: 0.8837890625
Batch: 147, Loss: 0.272170752286911, Accuracy: 0.9111328125
Batch: 148, Loss: 0.3305208683013916, Accuracy: 0.8876953125
Batch: 149, Loss: 0.3029525876045227, Accuracy: 0.880859375
Batch: 150, Loss: 0.28883472084999084, Accuracy: 0.904296875
Batch: 151, Loss: 0.31676244

Batch: 122, Loss: 0.3011564612388611, Accuracy: 0.9091796875
Batch: 123, Loss: 0.2696576416492462, Accuracy: 0.9140625
Batch: 124, Loss: 0.28237488865852356, Accuracy: 0.9052734375
Batch: 125, Loss: 0.32193607091903687, Accuracy: 0.8935546875
Batch: 126, Loss: 0.2829653024673462, Accuracy: 0.9052734375
Batch: 127, Loss: 0.2901870608329773, Accuracy: 0.896484375
Batch: 128, Loss: 0.3537246584892273, Accuracy: 0.8837890625
Batch: 129, Loss: 0.2809995114803314, Accuracy: 0.9130859375
Batch: 130, Loss: 0.33322376012802124, Accuracy: 0.8984375
Batch: 131, Loss: 0.29798802733421326, Accuracy: 0.8984375
Batch: 132, Loss: 0.3340405821800232, Accuracy: 0.8896484375
Batch: 133, Loss: 0.2930699586868286, Accuracy: 0.892578125
Batch: 134, Loss: 0.34119752049446106, Accuracy: 0.8857421875
Batch: 135, Loss: 0.3072925806045532, Accuracy: 0.896484375
Batch: 136, Loss: 0.3151906132698059, Accuracy: 0.8955078125
Batch: 137, Loss: 0.36290618777275085, Accuracy: 0.875
Batch: 138, Loss: 0.3029690682888031,

Batch: 109, Loss: 0.32204437255859375, Accuracy: 0.892578125
Batch: 110, Loss: 0.29422906041145325, Accuracy: 0.9091796875
Batch: 111, Loss: 0.3587103486061096, Accuracy: 0.869140625
Batch: 112, Loss: 0.3034399151802063, Accuracy: 0.8984375
Batch: 113, Loss: 0.31920456886291504, Accuracy: 0.884765625
Batch: 114, Loss: 0.37095320224761963, Accuracy: 0.8779296875
Batch: 115, Loss: 0.31624582409858704, Accuracy: 0.884765625
Batch: 116, Loss: 0.30316001176834106, Accuracy: 0.8974609375
Batch: 117, Loss: 0.3325548470020294, Accuracy: 0.8955078125
Batch: 118, Loss: 0.27216288447380066, Accuracy: 0.916015625
Batch: 119, Loss: 0.27448293566703796, Accuracy: 0.90625
Batch: 120, Loss: 0.29626739025115967, Accuracy: 0.904296875
Batch: 121, Loss: 0.28322288393974304, Accuracy: 0.9033203125
Batch: 122, Loss: 0.28444230556488037, Accuracy: 0.8974609375
Batch: 123, Loss: 0.3014588952064514, Accuracy: 0.9013671875
Batch: 124, Loss: 0.2856524586677551, Accuracy: 0.9052734375
Batch: 125, Loss: 0.3332000

Batch: 96, Loss: 0.30085861682891846, Accuracy: 0.8876953125
Batch: 97, Loss: 0.2291281372308731, Accuracy: 0.9150390625
Batch: 98, Loss: 0.33908605575561523, Accuracy: 0.884765625
Batch: 99, Loss: 0.3175331950187683, Accuracy: 0.8916015625
Batch: 100, Loss: 0.29451966285705566, Accuracy: 0.8974609375
Batch: 101, Loss: 0.29947346448898315, Accuracy: 0.8984375
Batch: 102, Loss: 0.29480862617492676, Accuracy: 0.904296875
Batch: 103, Loss: 0.36124444007873535, Accuracy: 0.8837890625
Batch: 104, Loss: 0.3087233304977417, Accuracy: 0.8837890625
Batch: 105, Loss: 0.3596062958240509, Accuracy: 0.875
Batch: 106, Loss: 0.2588586211204529, Accuracy: 0.923828125
Batch: 107, Loss: 0.31551048159599304, Accuracy: 0.8935546875
Batch: 108, Loss: 0.31855785846710205, Accuracy: 0.8876953125
Batch: 109, Loss: 0.3110659718513489, Accuracy: 0.8994140625
Batch: 110, Loss: 0.29594752192497253, Accuracy: 0.892578125
Batch: 111, Loss: 0.2709423899650574, Accuracy: 0.91015625
Batch: 112, Loss: 0.304259598255157

In [13]:
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log

,Epoch,Loss,Accuracy
0,1,2.985665,0.223633
1,2,2.093335,0.421875
2,3,1.724254,0.515625
3,4,1.548247,0.544922
4,5,1.420009,0.570312
...,...,...,...
95,96,0.305465,0.902344
96,97,0.316762,0.889648
97,98,0.262290,0.917969
98,99,0.272518,0.910156
